# <center> 🤿 Погружение в Агентов LangChain  </center>
### <center> 😎 Создаём "суперагента" 🕵️‍♂️ </center>

### Оглавление ноутбука
<img src='../images/ag2.webp' align="right" width="508" height="428" >
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#p1">🚀 Введение. 🦞 RAG  и как его готовить! </a></li>
<li><a href="#p2">📥 Document loader & Text Splitter ⚔️</a></li><ul type="square">
<li><a href="#p2.1">🚛 Document loaders </a></li>
<li><a href="#p2.2">🖖 Text splitters  </a></li></ul>
<li><a href="#p3">🔤 ➡️ 🔢 Embedding models </a></li><ul type="square">
<li><a href="#p3.1">Эмбеддинги от OpenAI (API) </a></li>
<li><a href="#p3.2">Эмбеддинги от HuggingFace 🤗</a></li></ul>
<li><a href="#p4">🗂 Vector Stores - место, где живут эмбеддинги </a></li>
<li><a href="#p5">🎣 Retrievers - выуди нужный документ </a></li>
<li><a href="#p6">🚰 RAG Pipeline - подключаем RAG к LLM 🔧 </a></li>
<li><a href="#p6">🧸 Выводы и заключения ✅ </a></li>


    
</ul></font></p>

## 🧑‍🎓 В этом уроке более подробно погрузимся в возможности агентов LangChain.

# <center> 🤖 Основные типы агентов и их суперспособности 🦾 </center>

<div class="alert alert-info">
    
📖 В `LangChain` представлены агенты 4-х основных типов (`Agent Types`):
* `ReAct (Reasoning and Acting)` - агенты такого типа создаются по умолчанию, работают с инструментами, принимающими один параметр на вход.
* `Structured chat` - могут работать с инструментами, которые принимают много параметров на вход
* `Self Ask` - работают по алгоритму SelfAsk только с инструментами поиска
* `XMLAgent, JSONAgent` - для работы с чат моделями в форматах XML и JSON
* Ещё есть `OpenAIFunctions Agents`, но OpenAI перестает их поддерживать. Им на замену пришли `OpenAI Tools Agents`, которые могут вызывать сразу одну или несколько функций, тем самым сокращая время для получения ответа - работает с последними моделями от OpenAI.

Рассмотрим подробнее первые три типа агентов. <br>
Каждый агент "под капотом" содержит тот или иной промпт завернутый в удобную обертку агента `LangChain`.

In [1]:
# !pip install langchain langchain-openai openai langchainhub google-search-results -q

In [1]:
import os
from getpass import getpass

In [160]:
# Для работы в колабе загрузите наш скрипт для использования ChatGPT с ключом курса, и необходимые файлы!
# !mkdir ../data/
# !wget -P ../data https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/data/PEP8.txt
# !wget -P ../data https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/data/Data.csv
# !wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [ ]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [2]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI

# course_api_key= "Введите ваш ключ, полученный в боте курса"
course_api_key = getpass(prompt="Введите ваш ключ, полученный в боте курса:")

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш ключ, полученный в боте курса: ········


# <center> ReAct Agent </center>

<img src='https://react-lm.github.io/files/diagram.png' align="right" width="608" height="428" >
<br>

Этот тип агента реализуется на основе техники `ReAct Prompting` представленной командой GoogleResearch в 2022 году в статье [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629). Собственно эта техника промптинга и легла в основу той концепции, которую мы сейчас называем автономными агентами.

Эта техника объединила в себе два подхода Reasoning (техника Chain of Thouts) и Acting (Action Plan Generation - генерация плана действий)


#### Посмотрим как выглядит шаблон промпта ReAct агента в `LangChain`:
```
Answer the following questions as best you can. You have access to the
following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}
```

Шаблоны промптов, используемые в агентах (и не только) хранятся в [LangChain Hub](https://smith.langchain.com/hub). Туда, примерно так же, как на гитхаб, можно загружать свои промпты и потом удобно ими пользоваться.

In [3]:
from langchain import hub

prompt = hub.pull("hwchase17/react")
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


### Инструмент поиск Google (SerpAPI)

Для использования поисковика Google по API необходимо зарегистрироваться на сайте [serpapi.com](https://serpapi.com) и получить API-ключ. На бесплатном тарифе доступно 100 запросов в месяц.

In [4]:
import langchain
from langchain.agents import AgentType, initialize_agent, load_tools, tool

In [5]:
# Добавляем полученный ключ в переменную среды
os.environ["SERPAPI_API_KEY"] = getpass(prompt="Введите ваш ключ SerpApi:")

Введите ваш ключ SerpApi: ········


In [6]:
# Легаси интерфейс и его недостатки
tools = load_tools(["serpapi"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose=True)
langchain.debug = True # включение Debug режима
#langchain.debug = False

agent.invoke('Какая температура в Сочи?')

/home/jovyan/.local/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Какая температура в Сочи?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Какая температура в Сочи?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat

{'input': 'Какая температура в Сочи?',
 'output': 'The current temperature in Sochi is 54 degrees Fahrenheit.'}

В легаси интерфейсе, не было явно видно какой именно промпт используется ддля агента и откуда он берется? Новый интерфейс создания агентов позволяет явно указывать промпт используемый в работе агента. <br>
Так же это открывает простой путь к кастомизации промптов для агентов, если дефолтные промпты не подходят для нашей конкретной задачи:
* Берем стандартный промпт, изменяем его. Или пишем собственный с нуля.
* Загружаем на `langchainhub`
* При необходимости подгружаем с хаба и пользуемся.

In [7]:
from langchain.agents import AgentExecutor, create_react_agent

tools = load_tools(["serpapi"], llm=llm)
langchain.debug = False

# В новом интерфейсе явно задается используемый промпт
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Температура в Сочи? Ответь в Цельсиях"})



> Entering new AgentExecutor chain...
I need to find the current temperature in Sochi in Celsius.
Action: Search
Action Input: "Sochi current temperature in Celsius"{'type': 'weather_result', 'temperature': '12', 'unit': 'Celsius', 'precipitation': '56%', 'humidity': '89%', 'wind': '3 km/h', 'location': 'Sochi, Russia', 'date': 'Wednesday 7:00 PM', 'weather': 'Cloudy'}The current temperature in Sochi is 12 degrees Celsius.
Final Answer: 12 degrees Celsius

> Finished chain.


{'input': 'Температура в Сочи? Ответь в Цельсиях',
 'output': '12 degrees Celsius'}

### <center> 2. Structured Chat Agent </center>

In [8]:
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: co

### 3. Self-Ask Agent
<img src='../images/self.webp' align="right" width="508" height="428" >

#### Посмотрим как выглядит шаблон промпта Self-Ask агента в `LangChain`:
```
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

Question: Are both the directors of Jaws and Casino Royale from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate answer: New Zealand.
So the final answer is: No

Question: {input}
Are followup questions needed here:{agent_scratchpad}
```

In [ ]:
from langchain.agents import create_self_ask_with_search_agent

prompt = hub.pull("hwchase17/self-ask-with-search")

## Агент с памятью 🧠

План:
1. параметры
2. Api для гугла от SerpApi
3. реакт + self ask with search
4. агент + память
5. агент + RAG (rag промпт from hub)
6. агент с sql базой (https://sqliteviewer.app - онлайн просмотр базы)
7. rest API + langsmith (продвинутые инструменты ?)
8. LangFlow

In [9]:
d = {18 : 1}

list(d.items())[0][0]

18